In [1]:
import requests
import pandas as pd

# Replace with your actual Census API key
API_KEY = 'c58cfcba458a97a78fa7f48cc758ccda5e3541cb'

# List of ZIP codes
# zip_codes = ['29715', '28205', '28120', '28207', '28001', '29708']  # Add all your ZIP codes here

file_path = "C:/Users/gangl/Documents/Python Scripts/Distance.csv"
df = pd.read_csv(file_path)

zip_codes = df['zipcode']

# Variables to retrieve
variables = {
    'B19013_001E': 'Median_Household_Income',
    'B19301_001E': 'Per_Capita_Income',
    'B17001_002E': 'Poverty_Count',
    'B17001_001E': 'Poverty_Universe',
    'B23025_005E': 'Unemployed',
    'B23025_003E': 'Labor_Force'
}

# Base URL for the API
base_url = 'https://api.census.gov/data/2022/acs/acs5'

# Prepare the list of variable keys
var_keys = ','.join(variables.keys())

# Initialize a list to store the results
data = []

for zip_code in zip_codes:
    params = {
        'get': var_keys,
        'for': f'zip code tabulation area:{zip_code}',
        'key': API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        result = response.json()
        headers = result[0]
        values = result[1]
        record = dict(zip(headers, values))
        # Calculate poverty rate and unemployment rate
        try:
            poverty_rate = (int(record['B17001_002E']) / int(record['B17001_001E'])) * 100
        except ZeroDivisionError:
            poverty_rate = None
        try:
            unemployment_rate = (int(record['B23025_005E']) / int(record['B23025_003E'])) * 100
        except ZeroDivisionError:
            unemployment_rate = None
        data.append({
            'ZIP Code': zip_code,
            'Median Household Income': record['B19013_001E'],
            'Per Capita Income': record['B19301_001E'],
            'Poverty Rate (%)': round(poverty_rate, 2) if poverty_rate is not None else None,
            'Unemployment Rate (%)': round(unemployment_rate, 2) if unemployment_rate is not None else None
        })
    else:
        print(f"Failed to retrieve data for ZIP code {zip_code}")

# Create a DataFrame and display the results
df = pd.DataFrame(data)
print(df)


     ZIP Code Median Household Income Per Capita Income  Poverty Rate (%)  \
0       29715                  103892             46566              4.58   
1       28205                   68418             44548             17.60   
2       28120                   71996             38666              9.42   
3       28207                  217656            180631              2.60   
4       28001                   54284             28719             16.87   
..        ...                     ...               ...               ...   
163     28277                  117369             64880              4.19   
164     28001                   54284             28719             16.87   
165     29708                  125677             57970              3.06   
166     28215                   61855             29352             15.62   
167     28277                  117369             64880              4.19   

     Unemployment Rate (%)  
0                     2.87  
1                

In [2]:
df.to_csv("FiveZipCensusData.csv", index=False)